<a href="https://colab.research.google.com/github/JasonManesis/Wildfire-Smoke-Detection/blob/main/Scripts/Wildfires_Detectron2_Model_Configuration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade watermark blackcellmagic

In [ ]:
%load_ext blackcellmagic

In [13]:
# Import some common libraries
import numpy as np
import pandas as pd
import random
import yaml 
import os, json, cv2, random, matplotlib
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import matplotlib.font_manager
import datetime
import pytz

In [ ]:
#Mount google drive.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Install detectron2.

!pip install pyyaml==5.1

# !pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html
# Install detectron2 that matches the above pytorch version, see https://detectron2.readthedocs.io/tutorials/install.html for instructions.

!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html

# After installation, you need to "restart runtime" in Colab. 

# Model Configuration

In [1]:
def datetime_stamp(time_zone = 'Europe/Athens'):  
  
    """Saves the given configuration in the output and session's directories.

    Args:
        time_zone (str): Name of the timezone in which we want the produced 
        date-time stamp.

    Returns: 
        current_date_and_time (str): The current date and time in 
        dd_mm_yyyy__hh_mm format, based on the given timezone.

    """

    timezone = pytz.timezone(time_zone)
    current_date_and_time = datetime.datetime.now(tz=timezone)
    current_date_and_time = current_date_and_time.strftime("%d_%m_%Y__%H_%M")

    return current_date_and_time

In [2]:
import detectron2
from detectron2.config import get_cfg
from detectron2.config.config import CfgNode as CN

In [3]:
cfg = get_cfg()
# If we want to change an already created configuration file.
#cfg.merge_from_file('/content/drive/MyDrive/configuration.yaml') 
train_dir = '/content/drive/MyDrive/Datasets/Wildfires_2_COCO/train' 
test_dir = '/content/drive/MyDrive/Datasets/Wildfires_2_COCO/test' 

In [5]:
def INPUT_config(cfg, train_dir=train_dir, test_dir=test_dir, data_augmentation=True):

    """Parametrizes the configuration of the model input.

    Args:
        cfg (detectron2.config.config.CfgNode): Configuration node for 
        parameterization.
        train_dir (str): Path of training images directory.
        test_dir (str): Path of training images directory.
        data_augmentation (bool): Whether to use or not data augmentation 
        during training.

    Returns: 
        _C (detectron2.config.config.CfgNode): Parameterized configuration 
        node of the input.

    """

    _C = cfg
    _C.INPUT = CN()
    
    for dir in (train_dir,test_dir):

        #Open .json
        json_2_open = dir + '/' + 'labels.json'
        f = open(json_2_open)
        dataset = json.load(f)
        f.close()

        images_df = pd.DataFrame(dataset['images'])
        min_height = images_df['height'].min()
        max_height = images_df['height'].max()
        min_width = images_df['width'].min()
        max_width = images_df['width'].max()
        
        if dir==train_dir:
            # Size of the smallest side of the image during training.(list)
            _C.INPUT.MIN_SIZE_TRAIN = [min(min_height,min_width),] 
            # Maximum size of the side of the image during training.
            _C.INPUT.MAX_SIZE_TRAIN = max(max_height,max_width) 
        else:
            # Size of the smallest side of the image during testing. 
            # Set to zero to disable resize in testing.
            _C.INPUT.MIN_SIZE_TEST = min(min_height,min_width) 
            # Maximum size of the side of the image during testing.
            _C.INPUT.MAX_SIZE_TEST = max(max_height,max_width) 
        
        if data_augmentation == True:
            # Mode for flipping images used in data augmentation during training, 
            # choose one of ["horizontal, "vertical", "none"].
            _C.INPUT.RANDOM_FLIP = "horizontal" 
            # `True` if cropping is used for data augmentation during training.
            _C.INPUT.CROP = CN({"ENABLED": True}) 
            # Cropping type. 
            # See documentation of `detectron2.data.transforms.RandomCrop` for explanation.
            _C.INPUT.CROP.TYPE = "relative_range" 
            # Size of crop in range (0, 1] if CROP.TYPE is "relative" 
            # or "relative_range" and in number of pixels if CROP.TYPE is "absolute".
            _C.INPUT.CROP.SIZE = [0.9, 0.9] 

    return _C

In [15]:
def BB_FPN_PG_configs(cfg):

    """Parametrizes the configuration of the Backbone, FPN and Proposal 
    generator modules.

    Args:
        cfg (detectron2.config.config.CfgNode): Configuration node for 
        parameterization.

    Returns: 
        _C (detectron2.config.config.CfgNode): Parameterized configuration node 
        of the of the Backbone, FPN and Proposal generator modules.

    """

    _C = cfg  

    # ------------------------------------------------------------------------ #
    # Backbone options
    # ------------------------------------------------------------------------ #
    _C.MODEL.BACKBONE = CN()

    _C.MODEL.BACKBONE.NAME = "build_resnet_backbone"
    # Freeze the first several stages so they are not trained.
    # There are 5 stages in ResNet. The first is a convolution, and the following
    # stages are each group of residual blocks.
    _C.MODEL.BACKBONE.FREEZE_AT = 2


    # ------------------------------------------------------------------------ #
    # FPN options
    # ------------------------------------------------------------------------ #
    _C.MODEL.FPN = CN()
    # Names of the input feature maps to be used by FPN
    # They must have contiguous power of 2 strides
    # e.g., ["res2", "res3", "res4", "res5"]
    _C.MODEL.FPN.IN_FEATURES = []
    _C.MODEL.FPN.OUT_CHANNELS = 256

    # Options: "" (no norm), "GN"
    _C.MODEL.FPN.NORM = ""

    # Types for fusing the FPN top-down and lateral features. 
    # Can be either "sum" or "avg"
    _C.MODEL.FPN.FUSE_TYPE = "sum"


    # ------------------------------------------------------------------------ #
    # Proposal generator options
    # ------------------------------------------------------------------------ #
    _C.MODEL.PROPOSAL_GENERATOR = CN()
    # Current proposal generators include "RPN", "RRPN" and "PrecomputedProposals"
    _C.MODEL.PROPOSAL_GENERATOR.NAME = "RPN"
    # Proposal height and width both need to be greater than MIN_SIZE
    # (a the scale used during training or inference)
    _C.MODEL.PROPOSAL_GENERATOR.MIN_SIZE = 0

    return _C

In [17]:
def Anchor_Generator_config(cfg):

    """Parametrizes the configuration of the anchor generator module.

    Args:
        cfg (detectron2.config.config.CfgNode): Configuration node for 
        parameterization.

    Returns: 
        _C (detectron2.config.config.CfgNode): Parameterized configuration node 
        of the of the anchor generator module.

    """

    _C = cfg
    _C.MODEL.ANCHOR_GENERATOR = CN()
    
    # The generator can be any name in the ANCHOR_GENERATOR registry
    _C.MODEL.ANCHOR_GENERATOR.NAME = "DefaultAnchorGenerator"

    # Anchor sizes (i.e. sqrt of area) in absolute pixels w.r.t. the network input.
    # Format: list[list[float]]. SIZES[i] specifies the list of sizes to use for
    # IN_FEATURES[i]; len(SIZES) must be equal to len(IN_FEATURES) or 1.
    # When len(SIZES) == 1, SIZES[0] is used for all IN_FEATURES.
    _C.MODEL.ANCHOR_GENERATOR.SIZES = [[8, 16, 32, 48, 64, 96, 128, 196, 256, 320, 384, 512]]

    # Anchor aspect ratios. For each area given in `SIZES`, anchors with different aspect
    # ratios are generated by an anchor generator.
    # Format: list[list[float]]. ASPECT_RATIOS[i] specifies the list of aspect ratios (H/W)
    # to use for IN_FEATURES[i]; len(ASPECT_RATIOS) == len(IN_FEATURES) must be true,
    # or len(ASPECT_RATIOS) == 1 is true and aspect ratio list ASPECT_RATIOS[0] is used
    # for all IN_FEATURES.
    _C.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.25, 0.5, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]]

    # Anchor angles.
    # list[list[float]], the angle in degrees, for each input feature map.
    # ANGLES[i] specifies the list of angles for IN_FEATURES[i].
    _C.MODEL.ANCHOR_GENERATOR.ANGLES = [0, 90]

    # Relative offset between the center of the first anchor and the top-left corner of the image
    # Value has to be in [0, 1). Recommend to use 0.5, which means half stride.
    # The value is not expected to affect model accuracy.
    _C.MODEL.ANCHOR_GENERATOR.OFFSET = 0.5

    return _C

In [18]:
def RPN_config(cfg):

    """Parametrizes the configuration of the Region Proposal Network model.

    Args:
        cfg (detectron2.config.config.CfgNode): Configuration node for 
        parameterization.

    Returns: 
        _C (detectron2.config.config.CfgNode): Parameterized configuration 
        node of the Region Proposal Network model.

    """
    
    _C = cfg
    _C.MODEL.RPN = CN()

    _C.MODEL.RPN.HEAD_NAME = "StandardRPNHead"  # used by RPN_HEAD_REGISTRY
    _C.MODEL.RPN.IOU_THRESHOLDS = [0.4, 0.7]
    _C.MODEL.RPN.IOU_LABELS = [0, -1, 1]
    # Number of regions per image used to train RPN
    _C.MODEL.RPN.BATCH_SIZE_PER_IMAGE = 512
    # Target fraction of foreground (positive) examples per RPN minibatch
    _C.MODEL.RPN.POSITIVE_FRACTION = 0.5
    # Options are: "smooth_l1", "giou"
    _C.MODEL.RPN.BBOX_REG_LOSS_TYPE = "smooth_l1"
    _C.MODEL.RPN.BBOX_REG_LOSS_WEIGHT = 1.0
    # Weights on (dx, dy, dw, dh) for normalizing RPN anchor regression targets
    _C.MODEL.RPN.BBOX_REG_WEIGHTS = (1.0, 1.0, 1.0, 1.0)
    # The transition point from L1 to L2 loss. Set to 0.0 to make the loss simply L1.
    _C.MODEL.RPN.SMOOTH_L1_BETA = 0.0
    _C.MODEL.RPN.LOSS_WEIGHT = 1.0
    # Number of top scoring RPN proposals to keep before applying NMS
    # When FPN is used, this is *per FPN level* (not total)
    _C.MODEL.RPN.PRE_NMS_TOPK_TRAIN = 12000
    _C.MODEL.RPN.PRE_NMS_TOPK_TEST = 6000
    # Number of top scoring RPN proposals to keep after applying NMS
    # When FPN is used, this limit is applied per level and then again to the union
    # of proposals from all levels
    # NOTE: When FPN is used, the meaning of this config is different from Detectron1.
    # It means per-batch topk in Detectron1, but per-image topk here.
    # See the "find_top_rpn_proposals" function for details.
    _C.MODEL.RPN.POST_NMS_TOPK_TRAIN = 2000
    _C.MODEL.RPN.POST_NMS_TOPK_TEST = 1000
    # NMS threshold used on RPN proposals
    _C.MODEL.RPN.NMS_THRESH = 0.6
    # Set this to -1 to use the same number of output channels as input channels.
    _C.MODEL.RPN.CONV_DIMS = [-1]
    
    return _C

In [19]:
def ROI_Heads_config(cfg):

    """Parametrizes the configuration of the ROI heads.

    Args:
        cfg (detectron2.config.config.CfgNode): Configuration node for 
        parameterization.

    Returns: 
        _C (detectron2.config.config.CfgNode): Parameterized configuration 
        node of the ROI heads.

    """
    _C = cfg
    _C.MODEL.ROI_HEADS = CN()

    _C.MODEL.ROI_HEADS.NAME = "Res5ROIHeads"
    # Number of foreground classes
    _C.MODEL.ROI_HEADS.NUM_CLASSES = 1
    # Names of the input feature maps to be used by ROI heads
    # Currently all heads (box, mask, ...) use the same input feature map list
    # e.g., ["p2", "p3", "p4", "p5"] is commonly used for FPN
    _C.MODEL.ROI_HEADS.IN_FEATURES = ["res4"]
    # IOU overlap ratios [IOU_THRESHOLD]
    # Overlap threshold for an RoI to be considered background (if < IOU_THRESHOLD)
    # Overlap threshold for an RoI to be considered foreground (if >= IOU_THRESHOLD)
    _C.MODEL.ROI_HEADS.IOU_THRESHOLDS = [0.5]
    _C.MODEL.ROI_HEADS.IOU_LABELS = [0, 1]
    # RoI minibatch size *per image* (number of regions of interest [ROIs])
    # Total number of RoIs per training minibatch =
    #   ROI_HEADS.BATCH_SIZE_PER_IMAGE * SOLVER.IMS_PER_BATCH
    # E.g., a common configuration is: 512 * 16 = 8192
    _C.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
    # Target fraction of RoI minibatch that is labeled foreground (i.e. class > 0)
    _C.MODEL.ROI_HEADS.POSITIVE_FRACTION = 0.25

    # Only used on test mode

    # Minimum score threshold (assuming scores in a [0, 1] range); a value chosen to
    # balance obtaining high recall with not having too many low precision
    # detections that will slow down inference post processing steps (like NMS)
    # A default threshold of 0.0 increases AP by ~0.2-0.3 but significantly slows down
    # inference.
    _C.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05
    # Overlap threshold used for non-maximum suppression (suppress boxes with
    # IoU >= this threshold)
    _C.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.5
    # If True, augment proposals with ground-truth boxes before sampling proposals to
    # train ROI heads.
    _C.MODEL.ROI_HEADS.PROPOSAL_APPEND_GT = True

    return _C

In [20]:
def Box_Head_config(cfg):

    """Parametrizes the configuration of the ROI box heads.

    Args:
        cfg (detectron2.config.config.CfgNode): Configuration node 
        for parameterization.

    Returns: 
        _C (detectron2.config.config.CfgNode): Parameterized configuration node 
        of the ROI box heads.

    """

    _C = cfg
    _C.MODEL.ROI_BOX_HEAD = CN()

    # C4 don't use head name option
    # Options for non-C4 models: FastRCNNConvFCHead,
    _C.MODEL.ROI_BOX_HEAD.NAME = "FastRCNNConvFCHead"
    # Options are: "smooth_l1", "giou"
    _C.MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_TYPE = "smooth_l1"
    # The final scaling coefficient on the box regression loss, used to balance the magnitude of its
    # gradients with other losses in the model. See also `MODEL.ROI_KEYPOINT_HEAD.LOSS_WEIGHT`.
    _C.MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_WEIGHT = 1.0
    # Default weights on (dx, dy, dw, dh) for normalizing bbox regression targets
    # These are empirically chosen to approximately lead to unit variance targets
    _C.MODEL.ROI_BOX_HEAD.BBOX_REG_WEIGHTS = (10.0, 10.0, 5.0, 5.0)
    # The transition point from L1 to L2 loss. Set to 0.0 to make the loss simply L1.
    _C.MODEL.ROI_BOX_HEAD.SMOOTH_L1_BETA = 0.0
    _C.MODEL.ROI_BOX_HEAD.POOLER_RESOLUTION = 14
    _C.MODEL.ROI_BOX_HEAD.POOLER_SAMPLING_RATIO = 0
    # Type of pooling operation applied to the incoming feature map for each RoI
    _C.MODEL.ROI_BOX_HEAD.POOLER_TYPE = "ROIAlignV2"

    _C.MODEL.ROI_BOX_HEAD.NUM_FC = 0
    # Hidden layer dimension for FC layers in the RoI box head
    _C.MODEL.ROI_BOX_HEAD.FC_DIM = 1024
    _C.MODEL.ROI_BOX_HEAD.NUM_CONV = 0
    # Channel dimension for Conv layers in the RoI box head
    _C.MODEL.ROI_BOX_HEAD.CONV_DIM = 256
    # Normalization method for the convolution layers.
    # Options: "" (no norm), "GN", "SyncBN".
    _C.MODEL.ROI_BOX_HEAD.NORM = ""
    # Whether to use class agnostic for bbox regression
    _C.MODEL.ROI_BOX_HEAD.CLS_AGNOSTIC_BBOX_REG = False
    # If true, RoI heads use bounding boxes predicted by the box head rather than proposal boxes.
    _C.MODEL.ROI_BOX_HEAD.TRAIN_ON_PRED_BOXES = False

    return _C

In [21]:
def ResNets_config(cfg):

    """Parametrizes the configuration of the ResNet network/s.

    Args:
        cfg (detectron2.config.config.CfgNode): Configuration node for parameterization.

    Returns: 
        _C (detectron2.config.config.CfgNode): Parameterized configuration node of the ResNet network/s.

    """

    _C = cfg 
    _C.MODEL.RESNETS = CN()

    # Note that parts of a resnet may be used for both the backbone and the head
    # These options apply to both

    _C.MODEL.RESNETS.DEPTH = 50
    _C.MODEL.RESNETS.OUT_FEATURES = ["res4"]  # res4 for C4 backbone, res2..5 for FPN backbone

    # Number of groups to use; 1 ==> ResNet; > 1 ==> ResNeXt
    _C.MODEL.RESNETS.NUM_GROUPS = 1

    # Options: FrozenBN, GN, "SyncBN", "BN"
    _C.MODEL.RESNETS.NORM = "FrozenBN"

    # Baseline width of each group.
    # Scaling this parameters will scale the width of all bottleneck layers.
    _C.MODEL.RESNETS.WIDTH_PER_GROUP = 64

    # Place the stride 2 conv on the 1x1 filter
    # Use True only for the original MSRA ResNet; use False for C2 and Torch models
    _C.MODEL.RESNETS.STRIDE_IN_1X1 = True

    # Apply dilation in stage "res5"
    _C.MODEL.RESNETS.RES5_DILATION = 1

    # Output width of res2. Scaling this parameters will scale the width of all 1x1 convs in ResNet
    # For R18 and R34, this needs to be set to 64
    _C.MODEL.RESNETS.RES2_OUT_CHANNELS = 256
    _C.MODEL.RESNETS.STEM_OUT_CHANNELS = 64

    # Apply Deformable Convolution in stages
    # Specify if apply deform_conv on Res2, Res3, Res4, Res5
    _C.MODEL.RESNETS.DEFORM_ON_PER_STAGE = [False, False, False, False]
    # Use True to use modulated deform_conv (DeformableV2, https://arxiv.org/abs/1811.11168);
    # Use False for DeformableV1.
    _C.MODEL.RESNETS.DEFORM_MODULATED = False
    # Number of groups in deformable conv.
    _C.MODEL.RESNETS.DEFORM_NUM_GROUPS = 1

    return _C

In [22]:
#Extract config properties
cfg = get_cfg()
Config = INPUT_config(cfg, train_dir=train_dir, test_dir=test_dir)
Config = BB_FPN_PG_configs(Config)
Config = Anchor_Generator_config(Config)
Config = RPN_config(Config)
Config = ROI_Heads_config(Config)
Config = Box_Head_config(Config)
Config = ResNets_config(Config)

In [23]:
def save_config(cfg,file_name):

    """Saves the given configuration in the output and session's directories.

    Args:
        cfg (detectron2.config.config.CfgNode): Model's configuration.
        file_name (str): Name for saving .yaml file (default is 'configuration.yaml').

    """
    saving_path = "/content/drive/MyDrive" + '/' + file_name
    
    #Save configuration to file.
    with open(saving_path, "w") as f:
        f.write(cfg.dump())   
    
    #Copy output configuration in the session's directory.
    !cp $saving_path $file_name
    
    #Check if the given configuration saved successfully.
    test_cfg = get_cfg()
    test_cfg.merge_from_file(file_name)
    if test_cfg:
        print("Model configuration saved successfully!")
        cfg.freeze()
    else: print("Error in model configuration saving")  

In [24]:
def inspect_yaml(configuration_dir):

    """Opens the input .yaml file and prints its key-value pairs.

    Args:
        configuration_dir (str): Saved configuration's directory.

    """
    stream = open(configuration_dir, 'r')
    dictionary = yaml.safe_load(stream)
    for key, value in dictionary.items():
        print('\n')
        print (key + " : " + str(value))

In [25]:
save_config(Config, file_name = 'configuration_' + datetime_stamp() + '.yaml')

Model configuration saved successfully!
